<a href="https://colab.research.google.com/github/JameTran/cyber-crossology/blob/main/Cool_Hackathon_Project_uwu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2018 The TensorFlow Hub Authors.

Licensed under the Apache License, Version 2.0 (the "License");

In [ ]:
# Copyright 2018 The TensorFlow Hub Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

## Setup

This section sets up the environment for access to the Universal Sentence Encoder on TF Hub and provides examples of applying the encoder to words, sentences, and paragraphs.

In [ ]:
%%capture
!pip3 install seaborn

In [ ]:
#@title Load the Universal Sentence Encoder's TF Hub module
from absl import logging

import tensorflow as tf

import tensorflow_hub as hub
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import seaborn as sns
from google.colab import files
import random as rand


module_url = "https://tfhub.dev/google/universal-sentence-encoder-large/5" #@param ["https://tfhub.dev/google/universal-sentence-encoder/4", "https://tfhub.dev/google/universal-sentence-encoder-large/5"]
model = hub.load(module_url)
print ("module %s loaded" % module_url)
def embed(input):
  return model(input)

# Semantic Textual Similarity Task Example

The embeddings produced by the Universal Sentence Encoder are approximately normalized. The semantic similarity of two sentences can be trivially computed as the inner product of the encodings.

In [ ]:
def plot_similarity(labels, features, rotation):
  corr = np.inner(features, features)
  sns.set(font_scale=1.2)
  g = sns.heatmap(
      corr,
      xticklabels=labels,
      yticklabels=labels,
      vmin=0,
      vmax=1,
      cmap="YlOrRd")
  g.set_xticklabels(labels, rotation=rotation)
  g.set_title("Semantic Textual Similarity")

def run_and_plot(messages_):
  message_embeddings_ = embed(messages_)
  plot_similarity(messages_, message_embeddings_, 90)

In [ ]:
def semantic_similarity(v1, v2):
  embed_v1 = embed(v1)
  embed_v2 = embed(v2)
  return np.inner(embed_v1, embed_v2)[0][0]

#method to determine if two sentences have a semantic similarity above a given target value
# @ target is a value between 0 and 1, with higher values being more semantically similar
def is_similar(v1, v2, target):
  if semantic_similarity(v1, v2) > target:
    return True
  else:
      return False

a = ["quick"]
b = ["fast"]

print(semantic_similarity(a, b)) 
print(is_similar(a, b, 0.1))



**REMEMBER TO UPLOAD THE FILE FROM LOCAL MACHINE** 

Code to open the word bank text file, and creating a new list with no apostrophes and spaces (for the crossword generation)

In [ ]:
with open('the_words_singular.txt') as f:
    lines = f.read().splitlines()
nospaces = []
for i in lines:
  a = i.replace(' ', '')
  a = a.replace("'", '')
  nospaces.append(a)


User Input Section. Takes in the user's 'starter' words, as well as the number of words they want in their crossword

In [ ]:
usr_input = []
while True:
  a = str(input("Please enter the starter words for your crossword! To quit, type 'q' "))
  if a == 'q':
    break
  usr_input.append(a)

cw_number = int(input("How many words would you like in your crossword?: "))


Long ass computation so we put it in another cell

In [ ]:
## Computes all the semantic similarities, stores them in a dict so we don't need to calculate them again
# I know this is awful I am so sorry T_T
embed_scores = []
for i in usr_input:
  a = [i]
  temp = {}
  for j in lines:
    b = [j]
    temp[j] = semantic_similarity(a, b)
    print("uwu")

  embed_scores.append(temp)
  
print(embed_scores[0])

In [ ]:

## Take number of words in crossword, divide by length of input to find how many similar words allocated to each starter word
len_input = len(usr_input)

number_allocated = []  #List to hold the number of similar words allocated to each input
base_num = int(cw_number/len_input)#Determines the base number of words allocated
remainder = cw_number % len_input 

#Determines how many numbers are left over. and distributes them amongst the first n elements until no more remainder is left
for i in range(len_input):
  if remainder != 0:
    number_allocated.append(base_num + 1)
    remainder-=1
  else:
    number_allocated.append(base_num)
print(number_allocated)


In [ ]:
## Iterates through dictionary to find most similar values
# testing for 3 values

count = 0
temp = []
for key, value in embed_scores[0].items():
  if count > 50:
    break
  if value > 0.58:
    temp2 = []
    for i in embed_scores:
      temp2.append(i.get(key))
    if all(j >= 0.3 for j in temp2) == True:
      a = [key, value]
      temp.append(a)
      count+=1
    else:
      continue


print(temp)
print(len(temp))


  

In [ ]:
## Grab 25 numbers, order them from highest score
top_embeds = []
def Sort_sub_list(sub_li):
  
    # reverse = None (Sorts in Ascending order)
    # key is set to sort using second element of 
    # sublist lambda has been used
    return(sorted(sub_li, key = lambda x: x[1], reverse=True))

top_embeds = Sort_sub_list(temp)
top_embeds_25 = top_embeds[:26]
print(top_embeds_25)